<a href="https://colab.research.google.com/github/Balatanov/deneme/blob/main/Veri_temizleme_split.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import json
import os

# Dosya isimlerini buraya yaz (Colab'e yüklediğin isimlerle aynı olmalı)
files_to_check = ['mat_sorulari_1.json', 'kimya_sorulari.json', 'fizik_sorulari.json']

# Modelin kendi kendine konuştuğu (hatalı) kelimeleri yakalamak için filtre
dirty_keywords = ["Hata kontrolü", "revize", "soruyu değiştir", "kurgu hatası", "şıklar güncellenmeli", "internal monologue"]

def analyze_dataset(file_path):
    print(f"\n{'='*10} DOSYA ANALİZİ: {file_path} {'='*10}")

    if not os.path.exists(file_path):
        print(f"❌ Dosya bulunamadı: {file_path}")
        return

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)

        total_questions = len(data)
        print(f"✅ Toplam Soru Sayısı: {total_questions}")

        # Detaylı Kontrol
        dirty_count = 0
        missing_fields = 0
        five_options_check = 0

        for i, item in enumerate(data):
            # 1. Alan Kontrolü
            if not all(key in item for key in ['question', 'options', 'rationale', 'correct']):
                missing_fields += 1

            # 2. Şık Sayısı Kontrolü
            if 'options' in item and len(item['options']) != 5:
                five_options_check += 1

            # 3. Kirli Veri (Hallucination) Kontrolü
            text_content = (item.get('rationale', '') + item.get('question', '')).lower()
            if any(keyword.lower() in text_content for keyword in dirty_keywords):
                print(f"⚠️  UYARI: {i+1}. soruda şüpheli ifade tespit edildi!")
                print(f"    İçerik: {item.get('rationale', '')[:100]}...") # İlk 100 karakteri göster
                dirty_count += 1

        print(f"\n📊 ÖZET RAPOR:")
        print(f"- Eksik alanı olan sorular: {missing_fields}")
        print(f"- 5 şıktan farklı olanlar : {five_options_check}")
        print(f"- Temizlenmesi gerekenler : {dirty_count}")

        if dirty_count == 0 and missing_fields == 0 and five_options_check == 0:
            print("🚀 Veri seti TERTEMİZ görünüyor!")

    except json.JSONDecodeError:
        print("❌ JSON Format Hatası! Dosya düzgün bir JSON formatında değil (Virgül veya parantez hatası olabilir).")

# Fonksiyonu çalıştır
for file in files_to_check:
    analyze_dataset(file)


========== DOSYA ANALİZİ: mat_sorulari_1.json ==========
❌ Dosya bulunamadı: mat_sorulari_1.json

========== DOSYA ANALİZİ: kimya_sorulari.json ==========
❌ Dosya bulunamadı: kimya_sorulari.json

========== DOSYA ANALİZİ: fizik_sorulari.json ==========
❌ JSON Format Hatası! Dosya düzgün bir JSON formatında değil (Virgül veya parantez hatası olabilir).


In [ ]:
import json
import re

files_to_fix = ['mat_sorulari_1.json', 'kimya_sorulari.json', 'fizik_sorulari.json']

def repair_json(file_path):
    print(f"🔧 Onarılıyor: {file_path}...")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # 1. Markdown temizliği (```json ve ``` kısımlarını sil)
        content = content.replace("```json", "").replace("```", "").strip()

        # 2. Yan yana yapıştırılmış listeleri birleştirme
        # Sorun: [...] [...] -> Çözüm: ..., ...
        # Regex ile: ] (boşluk veya yeni satır) [ desenini bulup virgül ile değiştiriyoruz
        content = re.sub(r'\]\s*\[', ',', content)

        # 3. Fazladan köşeli parantez kalıntılarını temizle (eğer iç içe girdiyse)
        # Bazen [[...]] olabilir, bunu düzeltmek daha riskli ama basit bir kontrol yapalım:
        # Eğer başta birden fazla [ varsa tek'e indir
        if content.startswith('[[') and not content.startswith('[[{'):
             # Eğer [[{ ile başlıyorsa belki array of array'dir dokunmayalım ama genelde hata olur.
             pass

        # 4. LaTeX Backslash düzeltmesi (Matematik için kritik!)
        # Tek backslash (\) JSON'da kaçış karakteridir. LaTeX komutları (\frac) bozulur.
        # Basit bir hack: Eğer \ karakterinden sonra " gelmiyorsa ve zaten \\ değilse, onu \\ yap.
        # Ancak bu regex çok karmaşık olabilir. Şimdilik sadece formatı düzeltip yüklemeyi deneyelim.

        try:
            data = json.loads(content)
            print(f"   ✅ Başarıyla onarıldı! (Basit temizlik yetti)")
        except json.JSONDecodeError as e:
            print(f"   ⚠️ Basit temizlik yetmedi, derin analiz yapılıyor... Hata: {e}")
            # Olası bir trailing comma hatası (listenin sonundaki fazladan virgül)
            content = re.sub(r',\s*\]', ']', content)
            try:
                data = json.loads(content)
                print(f"   ✅ Virgül hatası düzeltildi ve yüklendi!")
            except:
                print(f"   ❌ DÜZELTİLEMEDİ. Hata konumu: {e.pos}")
                print(f"   Hatalı kısımdaki metin: {content[e.pos-20:e.pos+20]}")
                return

        # Düzeltilmiş veriyi kaydet
        new_filename = file_path.replace(".json", "_fixed.json")
        with open(new_filename, 'w', encoding='utf-8') as f:
            json.dump(data, f, indent=2, ensure_ascii=False)

        print(f"   💾 Kaydedildi: {new_filename}")
        print(f"   📊 Toplam Soru Sayısı: {len(data)}")

    except FileNotFoundError:
        print(f"❌ Dosya bulunamadı: {file_path}")

# Çalıştır
for file in files_to_fix:
    repair_json(file)

🔧 Onarılıyor: mat_sorulari_1.json...
❌ Dosya bulunamadı: mat_sorulari_1.json
🔧 Onarılıyor: kimya_sorulari.json...
❌ Dosya bulunamadı: kimya_sorulari.json
🔧 Onarılıyor: fizik_sorulari.json...
   ✅ Başarıyla onarıldı! (Basit temizlik yetti)
   💾 Kaydedildi: fizik_sorulari_fixed.json
   📊 Toplam Soru Sayısı: 501


In [ ]:
import json
import re

files_to_fix = ['mat_sorulari_1.json', 'kimya_sorulari.json']

def aggressive_cleaner(file_path):
    print(f"☢️  Nükleer Temizlik Başlıyor: {file_path}...")

    try:
        with open(file_path, 'r', encoding='utf-8') as f:
            content = f.read()

        # ADIM 1: Markdown ve Birleştirme Hatalarını Temizle
        content = content.replace("```json", "").replace("```", "").strip()
        content = re.sub(r'\]\s*\[', ',', content) # ] [ şeklindeki kopuklukları birleştir

        # ADIM 2: Kimya Dosyası İçin LaTeX Düzeltici (Kritik!)
        # Mantık: Eğer \ işareti var ama sonrasında JSON için geçerli karakterler (" \ / b f n r t u) YOKSA
        # O zaman bu bir LaTeX komutudur (örn: \% veya \times). Bunu \\ yapmalıyız.
        # Regex: \ karakterini bul, eğer arkasından " \ / b f n r t u gelmiyorsa, onu \\ ile değiştir.
        content = re.sub(r'\\(?![/u"bfnrt\\])', r'\\\\', content)

        # ADIM 3: Matematik Dosyası İçin "Control Character" Temizliği
        # ASCII tablosunda 0-31 arasındaki (yeni satır ve tab hariç) görünmez karakterleri siler.
        # Bu, "Invalid control character" hatasını çözer.
        def clean_control_chars(match):
            char = match.group()
            return char if char in '\n\r\t' else '' # Enter ve Tab kalsın, diğer garip karakterler gitsin

        content = re.sub(r'[\x00-\x1f]', clean_control_chars, content)

        # ADIM 4: Yüklemeyi Dene
        try:
            data = json.loads(content)
            print("   ✅ İLK DENEMEDE BAŞARILI!")
        except json.JSONDecodeError as e:
            print(f"   ⚠️ Hâlâ hata var: {e}. Otomatik kırpma deneniyor...")

            # Eğer dosya yarım kaldıysa (örn: [Image of unit cir...) son süslü parantezi bulup oradan keselim.
            last_bracket = content.rfind('}')
            if last_bracket != -1:
                content_fixed = content[:last_bracket+1] + "\n]"
                try:
                    data = json.loads(content_fixed)
                    print("   ✅ DOSYA KIRPILARAK KURTARILDI! (Son soru eksik olabilir ama dosya çalışır)")
                except:
                    print("   ❌ Kurtarılamadı. Lütfen hatayı kontrol et.")
                    return
            else:
                return

        # Temizlenmiş veriyi kaydet
        output_name = file_path.replace(".json", "_ready.json")
        with open(output_name, 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"   💾 Kaydedildi: {output_name}")
        print(f"   📊 Toplam Soru Sayısı: {len(data)}")

    except FileNotFoundError:
        print(f"❌ Dosya bulunamadı: {file_path}")

# Çalıştır
for file in files_to_fix:
    aggressive_cleaner(file)

☢️  Nükleer Temizlik Başlıyor: mat_sorulari_1.json...
   ⚠️ Hâlâ hata var: Invalid \escape: line 1346 column 21 (char 67514). Otomatik kırpma deneniyor...
   ❌ Kurtarılamadı. Lütfen hatayı kontrol et.
☢️  Nükleer Temizlik Başlıyor: kimya_sorulari.json...
   ⚠️ Hâlâ hata var: Invalid \escape: line 4984 column 292 (char 263819). Otomatik kırpma deneniyor...
   ❌ Kurtarılamadı. Lütfen hatayı kontrol et.


In [ ]:
import json
import re

files_to_fix = ['mat_sorulari.json', 'kimya_sorulari.json']

def surgical_fix(file_path):
    print(f"🚑 Cerrahi Müdahale Başlıyor: {file_path}...")

    if not os.path.exists(file_path):
        print(f"❌ Dosya bulunamadı: {file_path}")
        return

    with open(file_path, 'r', encoding='utf-8') as f:
        content = f.read()

    # 1. Ön Temizlik (Markdown vs)
    content = content.replace("```json", "").replace("```", "").strip()
    content = re.sub(r'\]\s*\[', ',', content)

    # 2. Kontrol Karakterlerini Temizle (Görünmez bozucular)
    # Tab (\t) ve Yeni satır (\n, \r) hariç tüm 0-31 ASCII karakterlerini sil
    content = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f]', '', content)

    # 3. Akıllı LaTeX Kaçış Düzeltmesi (Regex ile)
    # Mantık: \ işaretinden sonra " \ / b f n r t u GELMİYORSA, o \ işaretini \\ yap.
    # Örnek: \alpha -> \\alpha olur. Ama \n -> \n kalır.
    content = re.sub(r'\\([^"\\/bfnrtu])', r'\\\\\1', content)

    # 4. Döngüsel Hata Düzeltme (Iterative Fix)
    # Eğer hala hata varsa, Python'un hatayı verdiği konuma gidip orayı manuel düzelteceğiz.
    max_retries = 10
    for attempt in range(max_retries):
        try:
            data = json.loads(content)
            print(f"   ✅ BAŞARILI! ({attempt}. denemede çözüldü)")

            # Başarılı olduysa kaydet
            output_name = file_path.replace(".json", "_final.json")
            with open(output_name, 'w', encoding='utf-8') as f:
                json.dump(data, f, ensure_ascii=False, indent=2)
            print(f"   💾 Kaydedildi: {output_name}")
            print(f"   📊 Soru Sayısı: {len(data)}")
            return

        except json.JSONDecodeError as e:
            print(f"   ⚠️ Hata: {e.msg} - Satır: {e.lineno}, Sütun: {e.colno}")

            # Hatanın olduğu konumu bul
            lines = content.split('\n')
            error_line_idx = e.lineno - 1

            if error_line_idx < len(lines):
                bad_line = lines[error_line_idx]
                print(f"      Hatalı Satır İçeriği: {bad_line.strip()[:50]}...")

                # ÇÖZÜM: O satırdaki TÜM tek backslashleri çiftle (tırnak içindekiler hariç demek zor, o yüzden kaba kuvvet)
                # Genellikle LaTeX içeren satır olduğu için oradaki \ -> \\ dönüşümü güvenlidir.
                # Ancak zaten düzgün olan \\n'i bozmamak lazım.

                # Basit çözüm: O satırdaki geçersiz kaçışları temizle
                # Python string'inde replace yapmak zor olduğu için regex kullanıyoruz yine ama daha agresif.

                new_line = re.sub(r'\\(?![/u"bfnrt\\])', r'\\\\', bad_line)

                # Eğer satır hiç değişmediyse ve hala hata veriyorsa, o satırda görünmez karakter veya tırnak hatası vardır.
                if new_line == bad_line:
                     # Son çare: Satırda tek kalan çift tırnak varsa onu escapele (JSON yapısını bozan tırnak)
                     # Ama bu çok riskli. Diğer seçenek: O karakteri silmek.
                     pass

                lines[error_line_idx] = new_line
                content = '\n'.join(lines)
            else:
                print("      ❌ Hata satırı dosya sınırları dışında!")
                break

    print("   ❌ Otomatik düzeltme başarısız oldu. Manuel kontrol gerekebilir.")
    # Yine de "kırpılmış" veya "düzeltilmiş" halini bir dosyaya yazalım ki görebil.
    with open(file_path.replace(".json", "_debug.txt"), 'w', encoding='utf-8') as f:
        f.write(content)
    print("      Son hali '_debug.txt' olarak kaydedildi.")

# Çalıştır
import os
for file in files_to_fix:
    surgical_fix(file)

🚑 Cerrahi Müdahale Başlıyor: mat_sorulari.json...
❌ Dosya bulunamadı: mat_sorulari.json
🚑 Cerrahi Müdahale Başlıyor: kimya_sorulari.json...
   ⚠️ Hata: Invalid \escape - Satır: 4984, Sütun: 292
      Hatalı Satır İçeriği: "rationale": "$AX_2E_2$ gösterimi merkez atomun 2 ...
   ⚠️ Hata: Invalid \escape - Satır: 5008, Sütun: 105
      Hatalı Satır İçeriği: "rationale": "Tam nötrleşme için $n(H^+) = n(OH^-)...
   ⚠️ Hata: Invalid \escape - Satır: 5032, Sütun: 43
      Hatalı Satır İçeriği: "rationale": "İdeal gaz denklemi $P \\\cdot V = n ...
   ⚠️ Hata: Invalid \escape - Satır: 5080, Sütun: 237
      Hatalı Satır İçeriği: "rationale": "Eşit kütle dendiğinde $M_A$'sı büyük...
   ✅ BAŞARILI! (4. denemede çözüldü)
   💾 Kaydedildi: kimya_sorulari_final.json
   📊 Soru Sayısı: 500


In [ ]:
import json
import os

# Dosya yolları
math_file = 'mat_sorulari.json'         # Orijinal (bozuk olan)
chem_file = 'kimya_sorulari_final.json' # Zaten kurtarmıştık

def clean_and_save_math():
    print("🧮 Matematik Verisi Hazırlanıyor...")
    try:
        with open(math_file, 'r', encoding='utf-8') as f:
            content = f.read()

        # 1. Temel Temizlik
        content = content.replace("```json", "").replace("```", "").strip()
        content = content.replace('] [', ',').replace('][', ',').replace(']\n[', ',')

        # 2. LaTeX Koruma (Placeholder Yöntemi)
        content = content.replace('\\\\', 'KORU_BS')
        content = content.replace('\\"', 'KORU_QT')
        content = content.replace('\\n', 'KORU_NL')
        content = content.replace('\\t', 'KORU_TB')

        # 3. Kalan tek backslashleri çiftle (LaTeX düzeltmesi)
        content = content.replace('\\', '\\\\')

        # 4. Korumayı Kaldır
        content = content.replace('KORU_BS', '\\\\')
        content = content.replace('KORU_QT', '\\"')
        content = content.replace('KORU_NL', '\\n')
        content = content.replace('KORU_TB', '\\t')

        data = json.loads(content)

        # Matematik dosyasını AYRI kaydet
        with open('matematik_final.json', 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"   ✅ MATEMATİK Hazır! ({len(data)} soru) -> 'matematik_final.json'")
        return True
    except Exception as e:
        print(f"   ❌ Matematik Hatası: {e}")
        return False

def check_and_save_chem():
    print("🧪 Kimya Verisi Hazırlanıyor...")
    try:
        with open(chem_file, 'r', encoding='utf-8') as f:
            data = json.load(f)

        # Kimya dosyasını (isminden emin olmak için) tekrar AYRI kaydet
        with open('kimya_final.json', 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"   ✅ KİMYA Hazır! ({len(data)} soru) -> 'kimya_final.json'")
        return True
    except Exception as e:
        print(f"   ❌ Kimya Hatası: {e}")
        return False

# İŞLEMİ BAŞLAT
if clean_and_save_math() and check_and_save_chem():
    print("\n🎉 İŞLEM TAMAM! İki ayrı uzman model için veriler ayrıştırıldı.")
    print("1. Model Eğitimi İçin: matematik_final.json")
    print("2. Model Eğitimi İçin: kimya_final.json")

🧮 Matematik Verisi Hazırlanıyor...
   ❌ Matematik Hatası: Invalid control character at: line 5177 column 363 (char 195533)


In [ ]:
import json
import re

math_file = 'mat_sorulari.json'

def deep_clean_math():
    print(f"🧮 Matematik Dosyası Derinlemesine Temizleniyor...")

    try:
        with open(math_file, 'r', encoding='utf-8') as f:
            content = f.read()

        # ADIM 1: Görünmez "Control Character" Temizliği (Hatanın asıl kaynağı)
        # Regex Açıklaması: ASCII 0-31 arası karakterleri bul.
        # ANCAK: \n (10), \r (13) ve \t (9) karakterlerini HARİÇ tut. Onlar JSON için lazımdır.
        # [\x00-\x08\x0b\x0c\x0e-\x1f\x7f] -> Bu aralıktaki her şeyi sil.
        content = re.sub(r'[\x00-\x08\x0b\x0c\x0e-\x1f\x7f]', '', content)

        # ADIM 2: Markdown ve Yapıştırma Hataları
        content = content.replace("```json", "").replace("```", "").strip()
        content = content.replace('] [', ',').replace('][', ',').replace(']\n[', ',')

        # ADIM 3: LaTeX Backslash Koruması (Placeholder Yöntemi)
        # Önce yasal kaçışları koru
        content = content.replace('\\\\', 'KORU_BS')
        content = content.replace('\\"', 'KORU_QT')
        content = content.replace('\\n', 'KORU_NL')
        content = content.replace('\\t', 'KORU_TB')

        # Kalan tek backslashleri çiftle (LaTeX düzeltmesi)
        content = content.replace('\\', '\\\\')

        # Korumayı kaldır
        content = content.replace('KORU_BS', '\\\\')
        content = content.replace('KORU_QT', '\\"')
        content = content.replace('KORU_NL', '\\n')
        content = content.replace('KORU_TB', '\\t')

        # ADIM 4: Yüklemeyi Dene
        try:
            data = json.loads(content)
        except json.JSONDecodeError as e:
            # Eğer dosya yarım kesildiyse (örn: internet koptuysa), son geçerli paranteze kadar kırp
            print(f"   ⚠️ Hâlâ format hatası var ({e}). Dosya sonunu onarıyorum...")
            last_bracket = content.rfind('}')
            if last_bracket != -1:
                content = content[:last_bracket+1] + "\n]"
                data = json.loads(content)
            else:
                raise e

        # Başarılıysa kaydet
        with open('matematik_final.json', 'w', encoding='utf-8') as f:
            json.dump(data, f, ensure_ascii=False, indent=2)

        print(f"   ✅ BAŞARDIK! Matematik verisi kurtarıldı.")
        print(f"   📊 Toplam Soru: {len(data)}")
        print(f"   📂 Dosya: matematik_final.json")

    except Exception as e:
        print(f"   ❌ Kurtarılamadı. Hata detayı: {e}")

# Çalıştır
deep_clean_math()

🧮 Matematik Dosyası Derinlemesine Temizleniyor...
   ⚠️ Hâlâ format hatası var (Invalid control character at: line 5177 column 363 (char 195533)). Dosya sonunu onarıyorum...
   ❌ Kurtarılamadı. Hata detayı: Invalid control character at: line 5177 column 363 (char 195533)


AYIRMA İŞLEMİ: 400 SORU TRAİN İÇİN 100 SORU TEST İÇİN

In [ ]:
import json
import random
import os

# Dosya isimleri
kaynak_dosya = "kimya_sorulari_fixed.json"
egitim_dosyasi = "kimya_train.json"
test_dosyasi = "kimya_test.json"

# 1. Veriyi Yükle
if os.path.exists(kaynak_dosya):
    with open(kaynak_dosya, 'r', encoding='utf-8') as f:
        data = json.load(f)

    toplam_soru = len(data)
    print(f"📂 Toplam Soru Sayısı: {toplam_soru}")

    # 2. Rastgele Karıştır (Her seferinde aynı karışım olsun diye seed sabitliyoruz)
    random.seed(42)
    random.shuffle(data)
    print("🔀 Sorular rastgele karıştırıldı.")

    # 3. Böl (400 Eğitim - Geriye Kalanlar Test)
    limit = 400

    if toplam_soru >= limit:
        train_data = data[:limit]      # İlk 400 soru
        test_data = data[limit:limit+100] # Sonraki 100 soru (varsa)

        # Eğer toplam 500'den azsa kalanı al
        if len(test_data) < 100:
            print(f"⚠️ Uyarı: 100 test sorusu çıkmadı, kalan {len(test_data)} soru test için ayrıldı.")

        # 4. Dosyaları Kaydet
        with open(egitim_dosyasi, 'w', encoding='utf-8') as f:
            json.dump(train_data, f, ensure_ascii=False, indent=4)

        with open(test_dosyasi, 'w', encoding='utf-8') as f:
            json.dump(test_data, f, ensure_ascii=False, indent=4)

        print("\n✅ İŞLEM BAŞARILI!")
        print(f"   📘 Eğitim Seti Oluştu: {egitim_dosyasi} ({len(train_data)} soru)")
        print(f"   📝 Test Seti Oluştu  : {test_dosyasi} ({len(test_data)} soru)")



    else:
        print(f"❌ HATA: Yeterli soru yok! En az 401 soru lazım ama sende {toplam_soru} var.")

else:
    print(f"❌ '{kaynak_dosya}' bulunamadı! Lütfen dosyayı yükle.")

📂 Toplam Soru Sayısı: 500
🔀 Sorular rastgele karıştırıldı.

✅ İŞLEM BAŞARILI!
   📘 Eğitim Seti Oluştu: kimya_train.json (400 soru)
   📝 Test Seti Oluştu  : kimya_test.json (100 soru)


In [ ]:
import json
import random
import os

# Dosya isimleri
kaynak_dosya = "mat_sorulari_fixed.json"
egitim_dosyasi = "mat_train.json"
test_dosyasi = "mat_test.json"

# 1. Veriyi Yükle
if os.path.exists(kaynak_dosya):
    with open(kaynak_dosya, 'r', encoding='utf-8') as f:
        data = json.load(f)

    toplam_soru = len(data)
    print(f"📂 Toplam Soru Sayısı: {toplam_soru}")

    # 2. Rastgele Karıştır (Her seferinde aynı karışım olsun diye seed sabitliyoruz)
    random.seed(42)
    random.shuffle(data)
    print("🔀 Sorular rastgele karıştırıldı.")

    # 3. Böl (400 Eğitim - Geriye Kalanlar Test)
    limit = 400

    if toplam_soru >= limit:
        train_data = data[:limit]      # İlk 400 soru
        test_data = data[limit:limit+100] # Sonraki 100 soru (varsa)

        # Eğer toplam 500'den azsa kalanı al
        if len(test_data) < 100:
            print(f"⚠️ Uyarı: 100 test sorusu çıkmadı, kalan {len(test_data)} soru test için ayrıldı.")

        # 4. Dosyaları Kaydet
        with open(egitim_dosyasi, 'w', encoding='utf-8') as f:
            json.dump(train_data, f, ensure_ascii=False, indent=4)

        with open(test_dosyasi, 'w', encoding='utf-8') as f:
            json.dump(test_data, f, ensure_ascii=False, indent=4)

        print("\n✅ İŞLEM BAŞARILI!")
        print(f"   📘 Eğitim Seti Oluştu: {egitim_dosyasi} ({len(train_data)} soru)")
        print(f"   📝 Test Seti Oluştu  : {test_dosyasi} ({len(test_data)} soru)")



    else:
        print(f"❌ HATA: Yeterli soru yok! En az 401 soru lazım ama sende {toplam_soru} var.")

else:
    print(f"❌ '{kaynak_dosya}' bulunamadı! Lütfen dosyayı yükle.")

📂 Toplam Soru Sayısı: 525
🔀 Sorular rastgele karıştırıldı.

✅ İŞLEM BAŞARILI!
   📘 Eğitim Seti Oluştu: mat_train.json (400 soru)
   📝 Test Seti Oluştu  : mat_test.json (100 soru)


In [ ]:
import json
import random
import os

# Dosya isimleri
kaynak_dosya = "fizik_sorulari_fixed.json"
egitim_dosyasi = "fizik_train.json"
test_dosyasi = "fizik_test.json"

# 1. Veriyi Yükle
if os.path.exists(kaynak_dosya):
    with open(kaynak_dosya, 'r', encoding='utf-8') as f:
        data = json.load(f)

    toplam_soru = len(data)
    print(f"📂 Toplam Soru Sayısı: {toplam_soru}")

    # 2. Rastgele Karıştır (Her seferinde aynı karışım olsun diye seed sabitliyoruz)
    random.seed(42)
    random.shuffle(data)
    print("🔀 Sorular rastgele karıştırıldı.")

    # 3. Böl (400 Eğitim - Geriye Kalanlar Test)
    limit = 400

    if toplam_soru >= limit:
        train_data = data[:limit]      # İlk 400 soru
        test_data = data[limit:limit+100] # Sonraki 100 soru (varsa)

        # Eğer toplam 500'den azsa kalanı al
        if len(test_data) < 100:
            print(f"⚠️ Uyarı: 100 test sorusu çıkmadı, kalan {len(test_data)} soru test için ayrıldı.")

        # 4. Dosyaları Kaydet
        with open(egitim_dosyasi, 'w', encoding='utf-8') as f:
            json.dump(train_data, f, ensure_ascii=False, indent=4)

        with open(test_dosyasi, 'w', encoding='utf-8') as f:
            json.dump(test_data, f, ensure_ascii=False, indent=4)

        print("\n✅ İŞLEM BAŞARILI!")
        print(f"   📘 Eğitim Seti Oluştu: {egitim_dosyasi} ({len(train_data)} soru)")
        print(f"   📝 Test Seti Oluştu  : {test_dosyasi} ({len(test_data)} soru)")



    else:
        print(f"❌ HATA: Yeterli soru yok! En az 401 soru lazım ama sende {toplam_soru} var.")

else:
    print(f"❌ '{kaynak_dosya}' bulunamadı! Lütfen dosyayı yükle.")

📂 Toplam Soru Sayısı: 501
🔀 Sorular rastgele karıştırıldı.

✅ İŞLEM BAŞARILI!
   📘 Eğitim Seti Oluştu: fizik_train.json (400 soru)
   📝 Test Seti Oluştu  : fizik_test.json (100 soru)


In [ ]:
import shutil
import os
from google.colab import drive

# 1. Drive'ı Bağla
drive.mount('/content/drive')

# 2. Hedef Klasör
hedef_klasor = "/content/drive/MyDrive/Proje_Yedekleri"

# Klasör yoksa oluştur (Garanti olsun)
os.makedirs(hedef_klasor, exist_ok=True)

# 3. Dosyaları Kopyala
dosyalar = ["kimya_train.json", "kimya_test.json"]

print("💾 Dosyalar Drive'a Yedekleniyor...")

for dosya in dosyalar:
    if os.path.exists(dosya):
        hedef_yol = os.path.join(hedef_klasor, dosya)
        shutil.copy(dosya, hedef_yol)
        print(f"   ✅ {dosya} -> Drive'a kopyalandı.")
    else:
        print(f"   ❌ {dosya} bulunamadı! (Önceki kodu çalıştırdın mı?)")

print("\n🎉 İşlem Tamam! Artık dosyaların güvende.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
💾 Dosyalar Drive'a Yedekleniyor...
   ❌ kimya_train.json bulunamadı! (Önceki kodu çalıştırdın mı?)
   ❌ kimya_test.json bulunamadı! (Önceki kodu çalıştırdın mı?)

🎉 İşlem Tamam! Artık dosyaların güvende.


In [ ]:
import shutil
import os
from google.colab import drive

# 1. Drive'ı Bağla
drive.mount('/content/drive')

# 2. Hedef Klasör
hedef_klasor = "/content/drive/MyDrive/Proje_Yedekleri"

# Klasör yoksa oluştur (Garanti olsun)
os.makedirs(hedef_klasor, exist_ok=True)

# 3. Dosyaları Kopyala
dosyalar = ["mat_train.json", "mat_test.json"]

print("💾 Dosyalar Drive'a Yedekleniyor...")

for dosya in dosyalar:
    if os.path.exists(dosya):
        hedef_yol = os.path.join(hedef_klasor, dosya)
        shutil.copy(dosya, hedef_yol)
        print(f"   ✅ {dosya} -> Drive'a kopyalandı.")
    else:
        print(f"   ❌ {dosya} bulunamadı! (Önceki kodu çalıştırdın mı?)")

print("\n🎉 İşlem Tamam! Artık dosyaların güvende.")

SÖZEL VERİLERİ ÇEKME İŞLEMLERİ

In [ ]:
import pandas as pd
from datasets import load_dataset
import json
import os
import numpy as np
from google.colab import drive

# 1. Drive Bağlantısı
drive.mount('/content/drive')
save_path = "/content/drive/MyDrive/Proje_Yedekleri/Sozel_Veriler"
os.makedirs(save_path, exist_ok=True)

print("🌍 Veri seti indiriliyor...")

try:
    dataset_dict = load_dataset("alibayram/turkish_mmlu", "default")
    all_data = []
    for split in dataset_dict.keys():
        all_data.append(dataset_dict[split].to_pandas())
    df_full = pd.concat(all_data, ignore_index=True)

    print(f"📚 Toplam Veri: {len(df_full)} satır.")

    # 2. ÖZEL FİLTRELEME FONKSİYONU
    def get_ygs_data(ders_adi, hedef_bolum, hedef_konu):
        print(f"\n🔍 {ders_adi} hazırlanıyor...")

        # 1. Bölüm Filtresi (Örn: 'YGS')
        # case=False: Küçük/büyük harf duyarsız (ygs, YGS, Ygs hepsi olur)
        mask_bolum = df_full['bolum'].astype(str).str.contains(hedef_bolum, case=False, na=False)
        subset = df_full[mask_bolum]
        print(f"   -> '{hedef_bolum}' bölümünde toplam {len(subset)} soru var.")

        # 2. Konu Filtresi (Örn: 'TÜRKÇE' veya 'FELSEFE')
        if hedef_konu:
            mask_konu = subset['konu'].astype(str).str.contains(hedef_konu, case=False, na=False)
            subset = subset[mask_konu]
            print(f"   -> '{hedef_konu}' konusu seçilince: {len(subset)} net soru kaldı.")

        return subset

    # --- FİLTRELEME AYARLARI ---

    # 1. EDEBİYAT -> Kaynak: YGS, Konu: TÜRKÇE
    df_edeb = get_ygs_data("Edebiyat", "YGS", "TÜRKÇE")

    # 2. FELSEFE -> Kaynak: YGS, Konu: FELSEFE
    df_fels = get_ygs_data("Felsefe", "YGS", "FELSEFE")

    # 3. DİN -> Kaynak: Geniş (Eski yöntem iyiydi)
    print("\n🔍 Din hazırlanıyor...")
    mask_din = df_full['bolum'].astype(str).str.contains('Din', case=False, na=False) | \
               df_full['konu'].astype(str).str.contains('Din', case=False, na=False)
    df_din = df_full[mask_din]
    print(f"   -> Din Kültürü toplam soru: {len(df_din)}")

    # --- KAYDETME DÖNGÜSÜ ---
    veriler = {"Edebiyat": df_edeb, "Felsefe": df_fels, "Din": df_din}

    for ders, df in veriler.items():
        if len(df) == 0:
            print(f"⚠️ HATA: {ders} için kriterlere uygun soru bulunamadı!")
            continue

        # 500 Tane Seç (veya ne kadar varsa)
        target = min(len(df), 500)
        df_final = df.sample(target, random_state=42)

        # Train (%80) / Test (%20)
        split = int(len(df_final) * 0.8)
        train_df = df_final.iloc[:split]
        test_df = df_final.iloc[split:]

        def format_json(dframe):
            res = []
            for _, row in dframe.iterrows():
                # Seçenekler (Numpy/String düzeltmesi)
                opts = row['secenekler']
                if isinstance(opts, np.ndarray): opts = opts.tolist()
                elif isinstance(opts, str):
                    try: opts = eval(opts)
                    except: opts = [opts]

                # Cevap
                raw = row['cevap']
                ans_map = {0:'A', 1:'B', 2:'C', 3:'D', 4:'E'}
                corr = ans_map.get(int(raw), 'A') if isinstance(raw, (int, np.integer)) else str(raw).upper()

                res.append({
                    "question": str(row['soru']),
                    "options": opts,
                    "correct": corr,
                    "rationale": f"Kaynak: {row['bolum']} - {row['konu']}", # Kaynağı da yazalım
                    "subject": ders
                })
            return res

        tr_json = format_json(train_df)
        ts_json = format_json(test_df)

        # Türkçe karakter destekli kayıt
        with open(f"{save_path}/{ders.lower()}_train.json", 'w', encoding='utf-8') as f:
            json.dump(tr_json, f, indent=4, ensure_ascii=False)
        with open(f"{save_path}/{ders.lower()}_test.json", 'w', encoding='utf-8') as f:
            json.dump(ts_json, f, indent=4, ensure_ascii=False)

        print(f"✅ {ders} OK: {len(tr_json)} Train / {len(ts_json)} Test")

    print("\n🎉 YGS ODAKLI VERİ SETLERİ HAZIR! (Edebiyat=Türkçe, Felsefe=Felsefe)")

except Exception as e:
    print(f"Hata: {e}")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
🌍 Veri seti indiriliyor...
📚 Toplam Veri: 279302 satır.

🔍 Edebiyat hazırlanıyor...
   -> 'YGS' bölümünde toplam 9789 soru var.
   -> 'TÜRKÇE' konusu seçilince: 1559 net soru kaldı.

🔍 Felsefe hazırlanıyor...
   -> 'YGS' bölümünde toplam 9789 soru var.
   -> 'FELSEFE' konusu seçilince: 1354 net soru kaldı.

🔍 Din hazırlanıyor...
   -> Din Kültürü toplam soru: 2250
✅ Edebiyat OK: 400 Train / 100 Test
✅ Felsefe OK: 400 Train / 100 Test
✅ Din OK: 400 Train / 100 Test

🎉 YGS ODAKLI VERİ SETLERİ HAZIR! (Edebiyat=Türkçe, Felsefe=Felsefe)
